
# Sense index calibration

## Objectives
- Show how Si responds to controlled changes in νf, ΔNFR, and phase alignment.
- Capture the deterministic ΔNFR/phase hook required by Phase-2 calibration harnesses.
- Provide reproducible Si readings that CI can compare verbatim to detect telemetry regressions.

## Phase-2 dependencies
- [Phase-2 integration notes](../fase2_integration.md) — the calibration harness reuses the scripted iterator pattern shown here.
- :mod:`tnfr.metrics.sense_index` — `compute_Si` is the canonical metric entry point for both runtime and analytical tooling.
- :mod:`tnfr.structural` — the operator sequence mirrors the canonical emission→reception→coherence→resonance→transition segment executed before adaptive selectors take over.

## Theoretical exposition
The sense index Si aggregates three forces: coherence (C), synchrony (phase alignment), and ΔNFR attenuation. Calibrating Si therefore requires scripted perturbations of νf, θ, and ΔNFR so that the resulting readings match expectations. By applying deterministic increments to each node we validate that Si increases for nodes receiving reinforced νf and tightened phase, and decreases otherwise. This preserves TNFR semantics for the Phase-2 calibration suite.

## Deterministic smoke check
The code instantiates a triad of nodes, computes baseline Si, then runs the canonical emission→reception→coherence→resonance→transition sequence while the hook iterates through predetermined ΔNFR, νf, and phase updates. The resulting before/after dictionary and per-step Si trace must remain stable; any variation signals a regression in Si aggregation or in the ΔNFR ledger that feeds it.


In [ ]:

from typing import Dict, Iterable, Tuple

from tnfr.constants import DNFR_PRIMARY, EPI_PRIMARY, THETA_PRIMARY, VF_PRIMARY
from tnfr.dynamics import set_delta_nfr_hook
from tnfr.metrics.sense_index import compute_Si
from tnfr.structural import Coherence, Emission, Reception, Resonance, Transition, create_nfr, run_sequence

G, anchor = create_nfr("calibration-anchor", epi=0.41, vf=1.1, theta=0.0)
_, fast = create_nfr("calibration-fast", graph=G, epi=0.37, vf=1.24, theta=0.18)
_, slow = create_nfr("calibration-slow", graph=G, epi=0.39, vf=0.96, theta=-0.32)
G.add_edge(anchor, fast)
G.add_edge(anchor, slow)
G.add_edge(fast, slow)

scripts: Dict[str, Iterable[Tuple[float, float, float]]] = {
    anchor: iter([(0.05, 0.015, 0.04), (0.03, 0.01, 0.02), (0.02, 0.008, 0.015), (0.015, 0.006, 0.01), (0.01, 0.004, 0.008)]),
    fast: iter([(0.07, 0.02, -0.03), (0.04, 0.015, -0.02), (0.03, 0.012, -0.015), (0.02, 0.008, -0.01), (0.015, 0.006, -0.008)]),
    slow: iter([(0.02, 0.0, 0.05), (0.015, 0.005, 0.03), (0.012, 0.004, 0.02), (0.01, 0.003, 0.015), (0.008, 0.002, 0.01)]),
}

baseline_si = {node: round(value, 6) for node, value in compute_Si(G, inplace=False).items()}
si_trace: list[dict[str, float]] = []


def calibration_hook(graph):
    for node_id, iterator in scripts.items():
        dnfr, vf_shift, theta_shift = next(iterator, (0.0, 0.0, 0.0))
        data = graph.nodes[node_id]
        vf = float(data[VF_PRIMARY])
        epi = float(data[EPI_PRIMARY])
        theta = float(data[THETA_PRIMARY])
        data[DNFR_PRIMARY] = dnfr
        data[EPI_PRIMARY] = epi + vf * dnfr
        data[VF_PRIMARY] = vf + vf_shift
        data[THETA_PRIMARY] = theta + theta_shift
    si_trace.append({node: round(val, 6) for node, val in compute_Si(graph, inplace=False).items()})

set_delta_nfr_hook(G, calibration_hook, note="sense index calibration smoke")
run_sequence(G, anchor, [Emission(), Reception(), Coherence(), Resonance(), Transition()])
calibrated_si = {node: round(value, 6) for node, value in compute_Si(G, inplace=False).items()}

{
    "baseline": baseline_si,
    "calibrated": calibrated_si,
    "per_step": si_trace,
}
